In [1]:
pip install  apache-tvm[tvmc]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 339.3/339.3 kB 35.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 678.1/678.1 kB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 88.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.9/125.9 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.6/173.6 MB 6.9 MB/s eta 0:00:00
  Using cached numpy-1.21.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (15.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of onnxruntime to determine which version is compatible with other requirements. This c

In [1]:
import tvm
print(tvm.__version__)

0.11.1


In [2]:
import numpy as np

In [18]:
import numpy as np
from PIL import Image
import os

def images_to_npz(directory, output_file):
    # List to hold image data
    images = []
    # Get all JPEG files from the directory
    for filename in os.listdir(directory):
        if filename.endswith('.jpg'):
            # Path to the image file
            filepath = os.path.join(directory, filename)
            # Open the image file
            with Image.open(filepath) as img:
                # Convert image to numpy array
                img_array = np.array(img)
                # Append to list of images
                images.append(img_array)

    # Save all images in a single .npz file
    np.savez_compressed(output_file, *images)

# Example usage
images_to_npz('/content/sample_data', 'output_images.npz')


In [9]:
#array= np.load('/content/sample_data/digit.npy')
#np.savez('digit.npz', arr_0= array)

In [10]:
#data= np.load('/content/sample_data/digit.npz')
#print(data['arr_0'])
#data.close

KeyError: 'arr_0 is not a file in the archive'

In [28]:
import numpy as np

# Load the NPZ file
data = np.load('/content/sample_data/output_images.npz')
input_tensor = data['arr_0']

# Now, prepare a dictionary with the correct input name expected by the model
model_inputs = {'data': input_tensor}


# List all arrays in the file
print("Arrays in NPZ file:", data.files)


# Access and print each array
for file in data.files:
    print(f"Contents of {file}:")
    print(data[file])
    print(f"shape of the input file '{file} : {data[file].shape}")


Arrays in NPZ file: ['arr_0']
Contents of arr_0:
[[[ 84  81  76]
  [ 84  81  76]
  [ 84  81  76]
  ...
  [ 69  81  41]
  [ 72  84  44]
  [ 75  87  47]]

 [[ 86  83  78]
  [ 86  83  78]
  [ 86  83  78]
  ...
  [ 73  85  45]
  [ 75  87  47]
  [ 78  90  50]]

 [[ 91  86  82]
  [ 92  87  83]
  [ 92  87  83]
  ...
  [ 74  86  46]
  [ 76  88  48]
  [ 78  90  50]]

 ...

 [[220 213 203]
  [229 222 212]
  [232 225 215]
  ...
  [109 106  73]
  [110 105  75]
  [108 103  74]]

 [[227 223 214]
  [230 226 215]
  [231 224 214]
  ...
  [104 106  69]
  [108 105  74]
  [105 101  72]]

 [[234 230 221]
  [235 231 222]
  [235 228 218]
  ...
  [102 104  67]
  [105 104  73]
  [104 100  71]]]
shape of the input file 'arr_0 : (360, 360, 3)


In [29]:

from PIL import Image

def prepare_image_array(array):
    # Convert array to Image for resizing
    image = Image.fromarray((array * 255).astype(np.uint8))  # Scale back to [0, 255] and convert to uint8
    #image = image.resize((224, 224))  # Resize image if necessary
    #array_resized = np.array(image).astype(np.float32) / 255.0  # Convert back to float32 and scale to [0, 1]

    image= image.convert('L')
    # Reshape to [ 28, 28]
    image= image.resize((28,28))
    array_resized = np.array(image).astype(np.float32) / 255.0

    # Add a batch dimension to make it [1, 1, 28, 28]
    array_resized = array_resized[np.newaxis, :]
    array_resized = array_resized[np.newaxis, :]
    return array_resized

# Process each image
for key in data.files:
    array = data[key]
    array_prepared = prepare_image_array(array)
    print(f"New shape of {key}:", array_prepared.shape)




New shape of arr_0: (1, 1, 28, 28)


In [24]:
pip install pillow

In [25]:
import tvm
from tvm.driver import tvmc

In [26]:
#load
model = tvmc.load("/content/sample_data/mnist-1.onnx", shape_dict={'Input73' : [1, 1, 28, 28]})


In [27]:
model.summary()

def @main(%Input73: Tensor[(1, 1, 28, 28), float32] /* ty=Tensor[(1, 1, 28, 28), float32] */) -> Tensor[(1, 10), float32] {
  %0 = divide(%Input73, 255f /* ty=float32 */) /* ty=Tensor[(1, 1, 28, 28), float32] */;
  %1 = nn.pad(%0, 0f /* ty=float32 */, pad_width=[[0i64, 0i64], [0i64, 0i64], [2i64, 2i64], [2i64, 2i64]]) /* ty=Tensor[(1, 1, 32, 32), float32] */;
  %2 = nn.conv2d(%1, meta[relay.Constant][0] /* ty=Tensor[(8, 1, 5, 5), float32] */, padding=[0, 0, 0, 0], channels=8, kernel_size=[5, 5]) /* ty=Tensor[(1, 8, 28, 28), float32] */;
  %3 = add(%2, meta[relay.Constant][1] /* ty=Tensor[(8, 1, 1), float32] */) /* ty=Tensor[(1, 8, 28, 28), float32] */;
  %4 = nn.relu(%3) /* ty=Tensor[(1, 8, 28, 28), float32] */;
  %5 = nn.max_pool2d(%4, pool_size=[2, 2], strides=[2, 2], padding=[0, 0, 0, 0]) /* ty=Tensor[(1, 8, 14, 14), float32] */;
  %6 = nn.pad(%5, 0f /* ty=float32 */, pad_width=[[0i64, 0i64], [0i64, 0i64], [2i64, 2i64], [2i64, 2i64]]) /* ty=Tensor[(1, 8, 18, 18), float32] */;
  %7 =

In [30]:
# Assuming 'data' is your loaded npz file
data = np.load("/content/sample_data/output_images.npz")

# Dictionary to store modified arrays
modified_arrays = {}

# Process each image and store in dictionary
for key in data.files:
    array = data[key]
    array_prepared = prepare_image_array(array)
    modified_arrays[key] = array_prepared  # Store the reshaped array
    print(f"New shape of {key}:", array_prepared.shape)

# Save the modified arrays back to an NPZ file
np.savez_compressed("/content/sample_data/modified_output_images.npz", **modified_arrays)

New shape of arr_0: (1, 1, 28, 28)


In [34]:
data= np.load("/content/sample_data/modified_output_images.npz")
# Extract the array stored under the key 'arr_0'
inputs = data['arr_0']  # This is the correct assignment

# Prepare the input dictionary expected by TVMC
# Replace 'arr_0' with the correct tensor name 'data'
minputs = {'Input73': inputs}



In [8]:
#tune
tvmc.tune(model,target="cuda", number = 2, tuning_records="tuning_logs.json")

RuntimeError: FLOP estimator fails for this operator. Error msg: The length of axis is not constant. . Please use `cfg.add_flop` to manually set FLOP for this operator

In [35]:
#compile
compiled_model= tvmc.compile(model, tuning_records="tuning_logs.json", target="llvm")

In [36]:
#Run
model_output= tvmc.run(compiled_model, device="cpu", inputs=minputs, profile=False)

2024-05-28 09:37:21.021 INFO load_module /tmp/tmphxa605ox/mod.so


In [37]:
model_output.save("/content/sample_data/output3.npz")

In [38]:
import numpy as np

# Load the NPZ file
data = np.load('/content/sample_data/output3.npz')

# List all arrays in the file and print their shape and a small preview
print("Arrays in the NPZ file and their previews:")
for key in data.files:
    print(f"Key: {key}")
    print("Shape:", data[key].shape)
    print("Data Preview (first few items):", data[key].flatten()[:10])  # Flatten and show first 10 items
    print()  # Add a newline for better readability between entries

# Close the file after processing
data.close()


Arrays in the NPZ file and their previews:
Key: output_0
Shape: (1, 10)
Data Preview (first few items): [-0.04485603  0.00779166  0.06810082  0.02999374 -0.12640963  0.14021875
 -0.0552849  -0.04938382  0.08432205 -0.05454041]

